<a href="https://colab.research.google.com/github/iamimpeccable/Deepfake-Detection-/blob/main/3dCnnDeepFake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

# Replace 'your_dataset.zip' with your uploaded zip file's name
with zipfile.ZipFile('/content/PreprocessedForVideo-20241024T231730Z-001.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')  # Extract everything into /content/dataset

# Ensure folders are extracted properly
preprocessed_folder1 = '/content/dataset/PreprocessedForVideo/Deepfakes'
preprocessed_folder2 = '/content/dataset/PreprocessedForVideo/original'
#os.listdir(preprocessed_folder1)
os.listdir(preprocessed_folder2)




['data_dst46', 'data_dst115', 'data_dst26', 'data_dst161', 'data_dst164']

In [ ]:
# List the contents of the PreprocessedForVideo folder
os.listdir(preprocessed_folder1)


['result26', 'result115', 'result46', 'result164', 'result161']

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Initialize lists to store image data and labels
image_data = []
labels = []

# Function to load images from subfolders
def load_images_from_folder(folder, label):
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)

        # Check if it's a directory (subfolder containing images)
        if os.path.isdir(subfolder_path):
            for image_name in os.listdir(subfolder_path):
                image_path = os.path.join(subfolder_path, image_name)

                # Read the image
                image = cv2.imread(image_path)
                if image is not None:
                    # Resize the image to 64x64 pixels (you can adjust this as needed)
                    image = cv2.resize(image, (64, 64))

                    # Convert to array and append to image data
                    image_data.append(img_to_array(image))
                    labels.append(label)

# Load images from both 'Deepfakes' and 'Original' folders
deepfake_folder = preprocessed_folder1
original_folder = preprocessed_folder2

# Load images for Deepfakes (label 1) and Originals (label 0)
load_images_from_folder(deepfake_folder, 1)
load_images_from_folder(original_folder, 0)

# Convert lists to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Normalize the images (scale pixel values between 0 and 1)
image_data = image_data / 255.0

# One-hot encode the labels
labels = to_categorical(labels, num_classes=2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

# Define 3D CNN model
model = models.Sequential()

# **Change 1: Reshape the input to have more frames in the temporal dimension**
# Reshape the input to add the temporal dimension, using at least 3 frames for 3D convolution
model.add(layers.Reshape((64, 64, 3, 1), input_shape=(64, 64, 3)))
# Reshape to (width, height, channels, frames) - This allows the Conv3D layer to work.
# Please note you might need to adjust this based on the desired temporal depth

# Add 3D Conv layer
model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling3D((2, 2, 1)))

# Add more Conv3D layers for better feature extraction
model.add(layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling3D((2, 2, 1)))

model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling3D((2, 2, 1)))

# Flatten the output of the Conv3D layers
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))  # 2 classes: Deepfake (1) and Original (0)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=13, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on the test set
y_pred = model.predict(X_test)

# Convert the predictions from one-hot encoding to labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test_labels, y_pred_labels))



Epoch 1/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 612s 4s/step - accuracy: 0.5036 - loss: 0.7231 - val_accuracy: 0.5553 - val_loss: 0.6884
Epoch 2/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 629s 4s/step - accuracy: 0.5324 - loss: 0.6914 - val_accuracy: 0.5580 - val_loss: 0.6895
Epoch 3/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 623s 4s/step - accuracy: 0.5187 - loss: 0.6927 - val_accuracy: 0.5553 - val_loss: 0.6900
Epoch 4/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 582s 4s/step - accuracy: 0.5263 - loss: 0.6920 - val_accuracy: 0.5553 - val_loss: 0.6864
Epoch 5/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 620s 4s/step - accuracy: 0.5199 - loss: 0.6932 - val_accuracy: 0.5274 - val_loss: 0.6884
Epoch 6/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 621s 4s/step - accuracy: 0.5348 - loss: 0.6880 - val_accuracy: 0.5606 - val_loss: 0.6832
Epoch 7/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 586s 4s/step - accuracy: 0.5403 - loss: 0.6816 - val_accuracy: 0.5633 - val_loss: 0.6584
Epoch 8/13
140/140 ━━━━━━━━━━━━━━━━━━━━ 582s 4s/step - accuracy: 0.6278 - loss: 0.6206 - val_accu